In [1]:
import pandas as pd
import numpy as np

import pyomo.environ as pyo

import gurobipy as gp
from gurobipy import GRB

import openpyxl

from icecream import ic


In [2]:
###
### RETRIEVE AND CLEAN DATA
###
file_path = "Railway services-2024.xlsx"

try:
    df = pd.read_excel(io=file_path, engine="openpyxl")
    ic(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")

# Data cleaning/ enhancing

# Removing unneccesary spaces from 'From' and 'To' columns. 
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

# Adding an indicator when the trip is located on line 400
indicator_line_400 = np.where(df['Line'] == 400.0, 1, 0)
df["line_400"] = indicator_line_400


ic| df.head():    Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line
               0     1       07:00:00     07:46:00    M  A        327         64   100
               1     2       07:00:00     07:57:00    H  M        936        308   800
               2     3       07:02:00     07:22:00    M  B        461        252   200
               3     4       07:04:00     08:03:00    M  J        428         10  1000
               4     5       07:06:00     08:13:00    M  F        449        124   600


In [3]:
### 
### CREATING THE MODEL
###

##
## Initial values:
##
trips = df['Trip'].to_list()
train_type = [
    "OC", 
    "OH"
    ]
cost_train_type = {
    "OC": 260,
    "OH": 210
}
length_train_type = {
    "OC": 100, 
    "OH": 70
    }
capacity_train_type = {
    "OC": 620, 
    "OH": 420
    }

#
# Data manipulated initial values:
#
cross_section = set(
    df['From'].values + df['To'].values
    ) #NOTE: Not used

trips_using_line400 = ((df.loc[df['line_400'] == 1]).index + 1).to_list() # +1 because the indexes start with 0, but the trips at 1. 


###
### Creation of the Model:
###
def create_general_model():
    m = pyo.ConcreteModel()

    ##
    ## CREATION OF SETS
    ##
    m.trips = pyo.Set(
        initialize = df['Trip']
        )

    m.train_type = pyo.Set(
        initialize = train_type
        )

    ##
    ## CREATION OF PARAMETERS
    ##
    m.cost_train_type = pyo.Param(
        m.train_type, 
        initialize = cost_train_type
    )

    m.length_train_type = pyo.Param(
        m.train_type, 
        initialize = length_train_type
        )

    m.capacity_train_type = pyo.Param(
        m.train_type, 
        initialize = capacity_train_type
        )
    
    #
    # Parameters created using functions:
    #
    m.trips_using_line400 = pyo.Param(
        m.trips,  # Assuming model.cross_section contains all possible cross sections
        initialize={trip: 1 if trip in trips_using_line400 else 0 for trip in m.trips}
    )

    m.passengers_per_trip = pyo.Param(
        m.trips, 
        initialize = {trip: df['Demand(μ)'].loc[trip - 1] for trip in m.trips} # -1 because the trips start with trip 1, but the df with index 0. So, loc[200] results in an error, because there are only 199 records. 
    )

    return m


m = create_general_model()


In [4]:
###
### ADDING SPECIFICS FOR MODEL1
###

##
## Variable specific for Model1
##
def specific_variables_Model1(m):
    #
    # Index set for allocation variable model1
    #
    m.index_set_allocation = pyo.Set(
        initialize = m.trips * m.train_type
        )

    #
    # Variable for allocation in Model1
    #
    # the variable represents the number of trains per type allocated to a trip.
    #
    m.allocation_train_numbers = pyo.Var(
        m.index_set_allocation, 
        domain = pyo.NonNegativeIntegers, 
        name = 'train_allocation', 
        doc = 'The number of trains of a certain type allocated to a cross-section'
    )
    return m

##
## Constraints specific for Model1
##
def specific_constraints_Model1(m):
    #
    # Rule for restricting the combined train length. 
    #
    def rule_maximum_length(
            m, 
            trip
            ):
        combined_train_length = sum(
            m.length_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return combined_train_length <= 300 - 100 * m.trips_using_line400[(trip)]
    m.constr_maximum_length = pyo.Constraint(
        m.trips, 
        rule = rule_maximum_length
        )
    
    #
    # Rule for enforcing the ability to accomodate all passengers. 
    #
    def rule_passenger_limit(
            m, 
            trip
            ):
        available_capacity = sum(
            m.capacity_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return available_capacity >= int(m.passengers_per_trip[(trip)])
    m.constr_passenger_limit = pyo.Constraint(
        m.trips, 
        rule = rule_passenger_limit
        )
    
    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 0.75 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            ) <= sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 0.75 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            ) <= sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    return m


##
## Objective rule specific for Model1
##
def specific_objective_Model1(m):
    #
    # Objective rule to minimize the total cost of all trains. 
    #
    def obj_minimize_total_cost(m):
        total_cost = sum(
            m.cost_train_type[(train_type)] * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips
            )
            for train_type in m.train_type
        )
        return total_cost
    m.objective = pyo.Objective(
        rule=obj_minimize_total_cost,
        sense=pyo.minimize
    )
    return m


##
## Add all variables, constraints and objectives to Model1
##
new_model = create_general_model()
model1 = specific_variables_Model1(m=new_model)
model1 = specific_constraints_Model1(m=model1)
model1 = specific_objective_Model1(m=model1)


In [5]:
###
### ADDING SPECIFICS FOR MODEL2
###


#
# Obtain all compositions
#
def get_compositions():
    # Define the lengths of each train type
    OH_length = 100
    OC_length = 70

    OH_cap = 420
    OC_cap = 620

    # Define the maximum length of a train
    max_train_length = 300

    # Generate all possible combinations of OH and OC trains
    all_combinations = {}

    for num_OH in range(max_train_length // OH_length + 1):
        for num_OC in range(max_train_length // OC_length + 1):

            length = num_OH * OH_length + num_OC * OC_length
            if length <= max_train_length:
                combination = []
                if num_OH > 0:  # Append 'OH' only if num_OH is greater than 0
                    combination.extend(['OH'] * num_OH)
                if num_OC > 0:  # Append 'OC' only if num_OC is greater than 0
                    combination.extend(['OC'] * num_OC)

                cap = num_OH * OH_cap + num_OC * OC_cap

                if length != 0:
                    all_combinations[str(combination)] = [length, cap, num_OC, num_OH]

    return all_combinations


#
# Obatin all allowed compositions per trip
#
def get_allowed_compositions_per_trip():
    composition_dict = get_compositions()
    compositions = composition_dict.keys()

    allowed_composition_per_trip = {}
    for trip in trips:
        
        demand = df['Demand(μ)'].loc[trip - 1]
        allowed_compositions = []

        for composition in compositions:
            
            if composition_dict.get(composition)[1] >= demand:
                
                if trip in trips_using_line400:
                    if composition_dict.get(composition)[0] <= 200:
                        allowed_compositions.append(composition)
                
                elif composition_dict.get(composition)[0] <= 300:
                    allowed_compositions.append(composition)

        allowed_composition_per_trip[trip] = allowed_compositions

    #ic(allowed_composition_per_trip)

    return allowed_composition_per_trip

##
## Parameter specific for Model2
##
def specific_parameter_Model2(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())

    #
    # Set containing the index for the quantification of the number of train types per composition.
    #
    m.index_quantify_compositions = pyo.Set(
        initialize= m.compositions * m.train_type
    )

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    m.quantify_compositions = pyo.Param(
        m.index_quantify_compositions,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in m.compositions for train_type in m.train_type
        }
    )

    return m


##
## Variable specific for Model2
##
def specific_variables_Model2(m):
    #
    # Index set for allocation variable model1
    #
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions)

    allowed = get_allowed_compositions_per_trip()
    m.allowed_composition_indicator = pyo.Param(
        m.trips * m.compositions,
        initialize={1 if comp in allowed.get(trip, []) else 0 for trip in m.trips for comp in m.compositions}
    )
    #m.allowed_composition_indicator.pprint()


    #
    # Variable for allocation in Model1
    #
    # the binary variable indicates the composition used on a trip.
    #
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip",
        doc = "Indicates which composition is used on a trip"
    )

    return m


##
## Constraints specific for Model2
##
def specific_constraints_Model2(m):
    #
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip):
        return sum(
            m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 0.75 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 0.75 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    allowed = get_allowed_compositions_per_trip()
    dfs = pd.DataFrame(index=df['Trip'], columns= get_compositions().keys())
    #ic(dfs)
    #param_data = []
    for trip in m.trips:
        for comp in m.compositions:
            value = 1 if comp in allowed.get(trip, []) else 0

            dfs.loc[trip, comp] = value
            #param_data.append({'Trip': trip, 'Comp': comp, 'Value': value})
    

    def rule_allowed_compositions(m, trip, composition):
        return m.allocation_compositions[(trip, composition)] <= dfs.loc[(trip, composition)]
    m.constr_allowed_compositions = pyo.Constraint(
        m.trips, 
        m.compositions,
        rule=rule_allowed_compositions
    )

    return m


def specific_objective_Model2(m):
    def obj_minimize_total_cost(m):
        return m.cost_train_type["OH"] * sum(
                m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            ) + m.cost_train_type["OC"] * sum(
                m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            )
    m.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)

    return m

new_model = create_general_model()
model2 = specific_parameter_Model2(m=new_model)
model2 = specific_variables_Model2(m=model2)
model2 = specific_constraints_Model2(m=model2)
model2 = specific_objective_Model2(m=model2)

implicit domain of 'Any'. The default domain for Param objects is 'Any'.
However, we will be changing that default to 'Reals' in the future.  If you
really intend the domain of this Paramto be 'Any', you can suppress this
warning by explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0) (called from
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-
packages/pyomo/core/base/indexed_component.py:804)


In [6]:
###
### CREATING THE SOLVING STRUCTURE
###


##
## Selecting the solver
##
solver = pyo.SolverFactory('cbc')


##
## Solve a specific model
##
def solve_model(model_to_solve, time_limit: int = 90, print_info: bool = True):
    #
    # Solve the model
    #
    results = solver.solve(
        model_to_solve, 
        options={'seconds': time_limit}
        )#, tee=True)

    #
    # Check the solver status
    #
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        print("Solver terminated successfully. Model is feasible.")
    elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
        print("Solver terminated: Model is infeasible.")
    else:
        print("Solver terminated with non-optimal solution.")

    #
    # Output the information from the solver if necessary. 
    #
    if print_info:
        ic(results)
        ic(results.solver.time)
    
    return results


##
## Obtain solution from Model1
##
def get_solution_df_Model1(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for train_type in model.train_type:
            solution_dict[(trip, train_type)] = model.allocation_train_numbers[(trip, train_type)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.train_type)

    for trip in model.trips:
        for train_type in model.train_type:
            solution_DF.loc[trip, train_type] = solution_dict.get((trip, train_type), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
      
        
    return solution_DF


##
## Obtain solution from Model2
##
def get_solution_df_Model2(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            solution_dict[(trip, composition)] = model.allocation_compositions[(trip, composition)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.compositions)

    for trip in model.trips:
        for composition in model.compositions:
            solution_DF.loc[trip, composition] = solution_dict.get((trip, composition), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
      
        
    return solution_DF




In [7]:
###
### Solving the actual models
###

#
# Solving Model1
#
#solve_model(model_to_solve=model1, time_limit=30)
#solution1 = get_solution_df_Model1(model=model1, display_solution_df=True)


#
# Solving Model2
#
solve_model(model_to_solve=model2, time_limit=30)
solution2 = get_solution_df_Model2(model=model2, display_solution_df=True)

#FIXME: The solutions are not the same.

ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 72690.0, 'Upper bound': 72690.0, 'Number of objectives': 1, 'Number of constraints': 198, 'Number of variables': 1509, 'Number of binary variables': 2000, 'Number of integer variables': 2000, 'Number of nonzeros': 1509, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.5, 'Wallclock time': 0.6, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 50, 'Number of created subproblems': 50}, 'Black box': {'Number of iterations': 234}}, 'Error rc': 0, 'Time': 0.6368498802185059}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 0.6368498802185059
ic| solution_DF:     ['OC'] ['OC', 'OC'] ['OC', 'OC', 'OC'] ['OC', 'OC', 'OC', 'OC'] ['OH']  \
               

Solver terminated successfully. Model is feasible.


                 196                0.0  
                 197                0.0  
                 198                0.0  
                 199                0.0  
                 200                0.0  
                 
                 [200 rows x 10 columns]
